In [18]:
!pip install langchain chromadb transformers google-cloud-aiplatform google-auth google-search-results > /dev/null

In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/content/generativeaitrial-trialLC.json'

In [2]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import VertexAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [3]:
from langchain.chains.summarize import load_summarize_chain

llm = VertexAI(temperature=0)
summary_chain = load_summarize_chain(llm, 
                                     chain_type="map_reduce")

In [4]:
from langchain.chains import AnalyzeDocumentChain

summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=summary_chain)

In [5]:
!unzip lc_documentdb.zip

Archive:  lc_documentdb.zip
   creating: content/lc_documentdb/
  inflating: content/lc_documentdb/chroma-embeddings.parquet  
   creating: content/lc_documentdb/index/
  inflating: content/lc_documentdb/index/uuid_to_id_b973adbd-91f7-4c95-b071-c5aabdbd5ab7.pkl  
  inflating: content/lc_documentdb/index/id_to_uuid_b973adbd-91f7-4c95-b071-c5aabdbd5ab7.pkl  
  inflating: content/lc_documentdb/index/index_b973adbd-91f7-4c95-b071-c5aabdbd5ab7.bin  
  inflating: content/lc_documentdb/index/index_metadata_b973adbd-91f7-4c95-b071-c5aabdbd5ab7.pkl  
  inflating: content/lc_documentdb/chroma-collections.parquet  


In [6]:
from langchain.embeddings import VertexAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

embeddings = VertexAIEmbeddings()

persist_directory = '/content/content/lc_documentdb'

vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embeddings)

db_retriever = vectordb.as_retriever()


langchain_qa = RetrievalQA.from_chain_type(llm=VertexAI(), 
                                 chain_type="stuff", 
                                 retriever=db_retriever)

In [16]:
def summarize_output(topic):
  relevant_data = langchain_qa(topic)
  #print(relevant_data)
  summary = summarize_document_chain.run(relevant_data['result'])
  return summary

In [8]:
query = 'List the langchain Agents'

In [ ]:
summarize_output(query)